In [ ]:
import numpy as np
from scipy.stats import rv_continuous
import matplotlib.pyplot as plt
from itertools import product
import pandas as pd

In [ ]:
# Reference for implementing a RV
# https://docs.scipy.org/doc/scipy/dev/contributor/adding_new.html#adding-a-new-statistics-distribution
# Similarly implemented RV: https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.gennorm.html#scipy.stats.gennorm

'''
Implementation with varying p

class special_exp(rv_continuous):
    def __init__(self, p, *args, **kwargs):
        self.p = p
        self.constant = 1 / (2*gamma(1+1/self.p))
        super().__init__(*args, **kwargs)
    
    def _pdf(self, x): #https://docs.scipy.org/doc/scipy/tutorial/stats/sampling.html
        return self.constant * np.exp(-np.abs(x)**self.p)
'''


class special_exp(rv_continuous): 
    def _pdf(self, x): #https://docs.scipy.org/doc/scipy/tutorial/stats/sampling.html
        return 1/2 * np.exp(-np.abs(x))
    
    def _cdf(self, x):
        if x >= 0:
            return 1/2 + 1/2 * (1-np.exp(-x))
        else:
            return 1/2 * np.exp(x)

    def _ppf(self, x):
        if x <= 1 and x >= 1/2:
            return -np.log(1-2*(x-1/2))
        else:
            return np.log(2*x)

def inv_cdf(x):
    if x >= 1/2:
        return -np.log(1-2*(x-1/2))
    else:
        return np.log(2*x)

# Generate g vector
def gen_g(n):
    g = []
    u = np.random.uniform(0,1,n)
    for u_i in u:
        g_i = inv_cdf(u_i)
        g.append(g_i)
    return g

# Generate Z
def gen_Z():
    return np.random.exponential(1, 1)

In [ ]:
# Generate random point in the diamond
def gen_point_in_diamond(n, alpha):
    g = np.array(gen_g(n))
    z = gen_Z()[0]
    x = []
    for g_i in g:
        x_i = g_i / (np.sum(abs(g)) + z)
        x.append(x_i)

    # Normalize volume
    x = alpha * np.array(x) * (gamma(1+n))**(1/n)/2

    return x

In [ ]:
# Generate random point in the cube
def gen_point_in_cube(dimension, bounds=[-1/2, 1/2]):
    x = []
    for i in range(dimension):
        x_i = np.random.uniform(bounds[0], bounds[1])
        x.append(x_i)
    return np.array(x)

In [ ]:
def get_pairwise_distances(points):
    m = len(points)
    #n = len(points[0])
    pairwise_distances = np.zeros(shape=(m, m))
    i = 0
    for point1 in points:
        j = 0
        for point2 in points:
            if i < j:
                pairwise_distances[i][j] = np.linalg.norm(np.array(point1) - np.array(point2))
            j += 1
        i += 1
        
    return pairwise_distances

In [ ]:
def check_for_contraction(pairwise_distances1, pairwise_distances2):
    upper_triangular = np.triu(np.ones_like(pairwise_distances1, dtype=bool),k=1)
    contraction = np.where(upper_triangular, pairwise_distances1 >= pairwise_distances2, True)
    if contraction.all():
        return True
    else:
        return False

In [ ]:
def estimate_volume_of_intersection(centers, n, r, n0=1000000):
    all_points = np.concatenate(centers)
    all_points = np.abs(all_points)
    extreme_point = np.max(all_points)
    bounds = np.array([-(extreme_point+r), extreme_point+r])
    samples = np.random.uniform(bounds[0], bounds[1], (n0, n))

    intersection = sum(all(np.linalg.norm(sample - center) <= r for center in centers) for sample in samples)
    volume_box = (bounds[1] - bounds[0])**n
    print(f"intersection: {intersection}")
    volume = (intersection / n0) * volume_box
    
    return volume

In [ ]:
def estimate_volume_of_intersection_grid(centers, n, r):
    n0 = 20**n
    num_step = int(np.ceil(n0**(1/n)))

    bounds = np.array([-(1+r), 1+r])
    steps = np.linspace(bounds[0], bounds[1], num_step)

    permutation = list(product(steps, repeat=n))
    grid = np.array(permutation)

    volume_box = (bounds[1] - bounds[0])**n

    intersection = sum(any(np.linalg.norm(sample - center) <= r for center in centers) for sample in grid)
    volume = (intersection / n0) * volume_box
    
    return (volume)**(1/n)

In [ ]:
def find_contraction(p, n, r, N, alpha):
    # Generate random points
    points_in_cube = []
    points_in_diamond = []
    for i in range(N):
        point_i = gen_point_in_cube(n)
        points_in_cube.append(point_i)
        point_j = gen_point_in_diamond(n, alpha)
        points_in_diamond.append(point_j)

    # Get pairwise distances and averages
    pairwise_distances_cube = get_pairwise_distances(points_in_cube)
    pairwise_distances_diamond = get_pairwise_distances(points_in_diamond)
    largest_avg_pairwise_distance = np.average(pairwise_distances_cube)
    smallest_avg_pairwise_distance = np.average(pairwise_distances_diamond)
    
    k = 1
    # Check for contraction
    if check_for_contraction(pairwise_distances_cube, pairwise_distances_diamond):
        cube_vol = estimate_volume_of_intersection_grid(points_in_cube, n, r)
        diamond_vol = estimate_volume_of_intersection_grid(points_in_diamond, n, r)
        return cube_vol, diamond_vol

    # Continue to generate new points while updated the largest and smallest pairwise distance averages
    while True:
        new_points_in_diamond = []
        new_points_in_cube = []
        for i in range(N):
            point_i = gen_point_in_cube(n)
            new_points_in_cube.append(point_i)
            point_j = gen_point_in_diamond(n, alpha)
            new_points_in_diamond.append(point_j)

        new_pairwise_distances_cube = get_pairwise_distances(new_points_in_cube)
        new_pairwise_distances_diamond = get_pairwise_distances(new_points_in_diamond)

        # Update largest
        cube_average = np.average(new_pairwise_distances_cube)
        if cube_average > largest_avg_pairwise_distance:
            points_in_cube = new_points_in_cube
            largest_avg_pairwise_distance = cube_average
            pairwise_distances_cube = new_pairwise_distances_cube

        # Update smallest
        diamond_average = np.average(new_pairwise_distances_diamond)
        if diamond_average < smallest_avg_pairwise_distance:
            points_in_diamond = new_points_in_diamond
            smallest_avg_pairwise_distance = diamond_average
            pairwise_distances_diamond = new_pairwise_distances_diamond
        
        
        k += 1
        
        # Check for contraction
        if check_for_contraction(pairwise_distances_cube, pairwise_distances_diamond):
            cube_vol = estimate_volume_of_intersection_grid(points_in_cube, n, r)
            diamond_vol = estimate_volume_of_intersection_grid(points_in_diamond, n, r)
            return cube_vol, diamond_vol

In [ ]:
def vary_r(n, r, N, alpha):
    df = pd.DataFrame(columns=['n', 'r', 'N', 'alpha', 'cube_vol', 'diamond_vol'])

    for radius in r:
        cube_vol, diamond_vol = find_contraction(1,n,radius,N,alpha)
        temp_df = pd.DataFrame({'n':[n],'r':[radius],'N':[N],'alpha':[alpha],'cube_vol':[cube_vol],'diamond_vol':[diamond_vol]})
        df = pd.concat([df, temp_df], ignore_index=True)
    
    return df

def vary_N(n, r, N, alpha):
    df = pd.DataFrame(columns=['n', 'r', 'N', 'alpha', 'cube_vol', 'diamond_vol'])

    for N0 in N:
        cube_vol, diamond_vol = find_contraction(1,n,r,N0,alpha)
        temp_df = pd.DataFrame({'n':[n],'r':[r],'N':[N0],'alpha':[alpha],'cube_vol':[cube_vol],'diamond_vol':[diamond_vol]})
        df = pd.concat([df, temp_df], ignore_index=True)

    return df

def vary_alpha(n, r, N, alpha):
    df = pd.DataFrame(columns=['n', 'r', 'N', 'alpha', 'cube_vol', 'diamond_vol'])

    for alph in alpha:
        cube_vol, diamond_vol = find_contraction(1,n,r,N,alpha)
        temp_df = pd.DataFrame({'n':[n],'r':[r],'N':[N],'alpha':[alph],'cube_vol':[cube_vol],'diamond_vol':[diamond_vol]})
        df = pd.concat([df, temp_df], ignore_index=True)

    return df


In [ ]:
def graph_r(df):
    n = df['n'][0]
    N = df['N'][0]
    alpha = df['alpha'][0]

    plt.plot(df['r'], df['cube_vol'], label='cube_vol', color='blue', marker='s')
    plt.plot(df['r'], df['diamond_vol'], label='diamond_vol', color='red', marker='D')

    plt.title(f'Varying r (n={n}, N={N}, alpha={alpha})')
    plt.xlabel('r')
    plt.ylabel('Vol (Normalized)')
    plt.grid()
    
    plt.legend()

def graph_N(df):
    n = df['n'][0]
    r = df['r'][0]
    alpha = df['alpha'][0]

    plt.plot(df['N'], df['cube_vol'], label='cube_vol', color='blue', marker='s')
    plt.plot(df['N'], df['diamond_vol'], label='diamond_vol', color='red', marker='D')

    plt.title(f'Varying N (n={n}, r={r}, alpha={alpha})')
    plt.xlabel('N')
    plt.ylabel('Vol (Normalized)')
    plt.grid()

    plt.legend()

def graph_alpha(df):
    n = df['n'][0]
    r = df['r'][0]
    N = df['N'][0]

    plt.plot(df['alpha'], df['cube_vol'], label='cube_vol', color='blue', marker='s')
    plt.plot(df['alpha'], df['diamond_vol'], label='diamond_vol', color='red', marker='D')

    plt.title(f'Varying alpha (n={n}, r={r}, N={N})')
    plt.xlabel('alpha')
    plt.ylabel('Vol (Normalized)')
    plt.grid()
    
    plt.legend()

In [ ]:
n = 6
r = [n**(1/4), n**(1/3), n**(1/2), n]
N = 12
alpha = .7
r_df = vary_r(n, r, N, alpha)
r_graph = graph_r(r_df)
plt.show(r_graph)

In [ ]:
n = 6
r = n**(1/2)
N = [8,10,12,14]
alpha = .7

N_df = vary_N(n, r, N, alpha)
N_graph = graph_N(N_df)
plt.show(N_graph)

In [ ]:
n = 6
r = n**(1/2)
N = 2*n
alpha = [.5, .55, .6, .65, .7, .75]
alpha_df = vary_alpha(n, r, N, alpha)
alpha_graph = graph_alpha(alpha_df)
plt.show(alpha_graph)